In [7]:
import chromadb
from chromadb.config import Settings

# Path to your Chroma DB (the folder where embeddings were saved)
CHROMA_PATH = "./chroma_store"   # <-- change if you used a different path

client = chromadb.PersistentClient(path=CHROMA_PATH)

# List all existing collections
collections = client.list_collections()

for c in collections:
    print("Collection Name:", c.name)


Collection Name: DBMS-25


In [8]:
# --- Imports ---
import os
import json
import numpy as np
import chromadb
from sentence_transformers import SentenceTransformer
from dotenv import load_dotenv
from sklearn.metrics.pairwise import cosine_similarity

load_dotenv()


True

In [9]:
# --- Embedding Model Config (768 dimensions) ---
model_name = "BAAI/bge-base-en-v1.5"   # 768-dim model (matches Chroma DB)
embedder = SentenceTransformer(model_name)


model.safetensors:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [10]:
# --- Connect to Chroma ---
client = chromadb.PersistentClient(path="./chroma_store")
collection = client.get_collection("DBMS-25")

print("Connected to Chroma. Dimension:",
      collection._client._get_collection(collection.id)["dimension"])


Connected to Chroma. Dimension: 768


In [11]:
# --- Query Configuration ---
query = "web scraping project using python"
TOP_K = 5

query_emb = embedder.encode(query).tolist()   # returns 768-dim vector


In [12]:
# --- Query Chroma ---
res = collection.query(
    query_embeddings=[query_emb],
    n_results=TOP_K,
    include=["metadatas", "documents", "distances", "embeddings"]
)

ids = res["ids"][0]
docs = res["documents"][0]
metas = res["metadatas"][0]
dists = res["distances"][0]


Exception occurred invoking consumer for subscription 07e7552f7ae14e7e8771f9bda941d4deto topic persistent://default/default/0ff97bea-3a0f-40ca-bae6-d6297f827339 Dimensionality of (768) does not match indexdimensionality (1024)
Exception occurred invoking consumer for subscription 07e7552f7ae14e7e8771f9bda941d4deto topic persistent://default/default/0ff97bea-3a0f-40ca-bae6-d6297f827339 Dimensionality of (768) does not match indexdimensionality (1024)
Exception occurred invoking consumer for subscription 07e7552f7ae14e7e8771f9bda941d4deto topic persistent://default/default/0ff97bea-3a0f-40ca-bae6-d6297f827339 Dimensionality of (768) does not match indexdimensionality (1024)
Exception occurred invoking consumer for subscription 07e7552f7ae14e7e8771f9bda941d4deto topic persistent://default/default/0ff97bea-3a0f-40ca-bae6-d6297f827339 Dimensionality of (768) does not match indexdimensionality (1024)
Exception occurred invoking consumer for subscription 07e7552f7ae14e7e8771f9bda941d4deto top

ValueError: shapes (1024,) and (768,) not aligned: 1024 (dim 0) != 768 (dim 0)

In [7]:
# --- Show Results ---
for i in range(TOP_K):
    print("="*60)
    print(f"Rank {i+1}")
    print(f"Distance: {dists[i]}")
    print(f"Title: {metas[i].get('title','')}")
    print(f"Snippet: {docs[i][:200]}...")



=== 🎯 Unique Top-5 Results After Deduplication ===

1. Scraping laptop data from Amazon
   Domain: Artificial Intelligence / Data Science
   Source: ISE-dept
   Tech Stack: Python 3.9, BeautifulSoup4, requests, pandas, NumPy
   Whole Similarity: 58.13%
   Title Sim: 61.03%, Desc Sim: 58.13%
   Snippet: Scraping laptop data from Amazon - This project aims to use Python libraries to scrape and extract data on laptop models, features, and pricing from A...

2. Beautiful Soup for Web Scraping
   Domain: Artificial Intelligence / Data Science
   Source: ISE-dept
   Tech Stack: Python, BeautifulSoup, lxml, requests, HTML, XML
   Whole Similarity: 34.46%
   Title Sim: 40.56%, Desc Sim: 34.46%
   Snippet: Beautiful Soup for Web Scraping - Explore the application of using this Python package for extracting data through the parsing of XML documents and HT...

3. Building an E-Commerce Clothing Classifier Model with Keras
   Domain: Artificial Intelligence / Data Science
   Source: ISE-dept
   T

In [8]:
# --- Utility ---
def to_percent(x):
    return round((1 - x) * 100, 2)  # turn cosine distance into a relevance %


In [9]:
# --- Deduplicate same titles ---
unique = {}
final_results = []

for i in range(len(ids)):
    title = metas[i]["title"]
    if title not in unique:
        unique[title] = True
        final_results.append({
            "title": title,
            "similarity": to_percent(dists[i]),
            "snippet": docs[i][:300],
            "metadata": metas[i]
        })
    if len(final_results) == 5:
        break

final_results



🧠 Generating synthesized explanation


=== 🔍 Top-5 Results ===

1. Scraping laptop data from Amazon
   Domain: Artificial Intelligence / Data Science | Whole Sim: 58.13%
   Field Sims -> Title: 61.03%, Desc: 58.13%, Tech: 21.47%
   Snippet: Scraping laptop data from Amazon - This project aims to use Python libraries to scrape and extract data on laptop models, features, and pricing from A...

2. Beautiful Soup for Web Scraping
   Domain: Artificial Intelligence / Data Science | Whole Sim: 34.46%
   Field Sims -> Title: 40.56%, Desc: 34.46%, Tech: 27.55%
   Snippet: Beautiful Soup for Web Scraping - Explore the application of using this Python package for extracting data through the parsing of XML documents and HT...

3. Building an E-Commerce Clothing Classifier Model with Keras
   Domain: Artificial Intelligence / Data Science | Whole Sim: 32.84%
   Field Sims -> Title: 33.38%, Desc: 32.84%, Tech: 7.92%
   Snippet: Building an E-Commerce Clothing Classifier Model with Keras - The Bui

In [ ]:
# --- Build Prompt for LLM ---
prompt_lines = [
    "You are an expert AI assistant.",
    "Here are the top retrieved projects:\n"
]

for r in final_results:
    prompt_lines.append(f"Title: {r['title']}")
    prompt_lines.append(f"Relevance: {r['similarity']}%")
    prompt_lines.append(f"Snippet: {r['snippet']}\n")

prompt_lines.append("\nGenerate a clear explanation and insights based on these results.")

llama_prompt = "\n".join(prompt_lines)
print(llama_prompt)


In [4]:
# --- Generate with Gemini ---
import google.generativeai as genai

genai.configure(api_key=os.getenv("GEMINI_API_KEY"))

response = genai.GenerativeModel("gemini-pro").generate_content(llama_prompt)
print(response.text)


NameError: name 'llama_prompt' is not defined

In [ ]:
import google.generativeai as genai

# --- Load env variables ---
load_dotenv()
GOOGLE_API_KEY = os.getenv("GEMINI_API_KEY")
MODEL_NAME = os.getenv("MODEL_NAME", "gemini-2.5-pro")

# --- Configure Gemini ---
genai.configure(api_key=GOOGLE_API_KEY)
model = genai.GenerativeModel(MODEL_NAME)

# --- Your prompt (already prepared earlier) ---
print("\n🧠 Generating synthesized explanation...")

response = model.generate_content(llama_prompt)   # reuse same prompt structure
ai_summary = response.text.strip()

# --- Display Results ---
print("\n\n=== 🔍 Top-5 Results ===")
for i, r in enumerate(final_results, start=1):
    print(f"\n{i}. {r['title']}")
    print(f"   Domain: {r['domain']} | Whole Sim: {to_percent(r['sim_whole'])}%")
    print(f"   Field Sims → Title: {to_percent(r['sim_title'])}%, "
          f"Desc: {to_percent(r['sim_description'])}%, "
          f"Tech: {to_percent(r['sim_tech_stack'])}%")
    print(f"   Snippet: {r['doc_snippet'][:150]}...")

print("\n\n=== 🧠 AI Explanation & Suggestions  ===\n")
print(ai_summary)

# --- Optional: Save to JSON ---
import json
out_path = f"analysis_reports/{query.replace(' ','_')}_gemini.json"
os.makedirs("analysis_reports", exist_ok=True)
with open(out_path, "w", encoding="utf-8") as f:
    json.dump({
        "query": query,
        "results": final_results,
        "ai_analysis": ai_summary
    }, f, indent=2, ensure_ascii=False)

print(f"\n✅ Saved Gemini analysis to: {out_path}")
